In [1]:
from ms_autoencoder import GraphEncoder
from sklearn.datasets import load_wine
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.cluster import normalized_mutual_info_score
from torch import nn, optim
import torch
import numpy as np
from tqdm import tqdm


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/Users/mustafaarif/opt/anaconda3/envs/networks/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Preprocessing

data = load_wine()
X = data.data
Y = data.target
k = len(np.unique(Y))

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)

S = cosine_similarity(X, X)

#Why multiply by D^-1 twice??
D = np.diag(1.0 / np.sqrt(S.sum(axis=1)))
X_train = torch.tensor(D.dot(S).dot(D)).float().to(device)

#Model Setup
layers = [len(X_train)] + [128, 64, 128] + [len(X_train)]
model = GraphEncoder(layers, k).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
#Training
epochs = 200
beta = 0.01
rho = 0.5


for epoch in range(1, epochs + 1):
    optimizer.zero_grad()
    print(epoch)
    X_hat = model(X_train)
    print(epoch)
    loss = model.loss(X_hat, X_train, beta, rho)
    nmi = normalized_mutual_info_score(model.get_cluster(), Y, average_method='arithmetic')

    loss.backward()
    optimizer.step()

    tq.set_postfix(loss='{:.3f}'.format(loss), nmi='{:.3f}'.format(nmi))
    tq.update()
print(model.get_cluster())